In [ ]:
import os
import sys

import pandas as pd
pd.options.display.max_columns = 999
from pyjarowinkler import distance

import sqlalchemy as sqla
from sqlalchemy import create_engine

DB_URI = os.getenv('CD_DWH')
engine = create_engine(DB_URI)

Jaro-Winkler distance metric was developed for name matching for the US census. Listed in environment.yaml for conda environment or `pip install pyjarowinkler`

The below SQL query extracts first name and last name from each of the name columns in the two tables.

We wish to calculate the Jaro-Winkler distance for each of the names. 

We could limit the number of comparisons by restricting to those that have the same initials with

`...
from candidate_donations D join election_results E
on substring(D.FirstName,1,1) = substring(E.FirstName,1,1)
and substring(D.LastName,1,1) = substring(E.LastName,1,1)`

In [ ]:
QUERY = """
with candidate_donations as
(
select
    lower(trim(regexp_replace(split_part(recipient_candidate_name, ', ', 2),'\s.(\.|$)',''))) as FirstName,
    lower(trim(split_part(recipient_candidate_name, ', ', 1))) as LastName,
    recipient_candidate_name as candidate_name,
    sum(transaction_amount) as total_transaction
from trg_analytics.candidate_contributions
where election_cycle = '2015'
group by recipient_candidate_name
),

election_results as
(
select *,
    lower(trim(regexp_replace(substring(candidate_name, 0, position(LastName in lower(candidate_name))),'\s.(\.|$)',''))) as FirstName from 
    (select contest_name, candidate_name, vote_total,
        rank() over (partition by contest_name order by vote_total desc) = 1 as is_winner,
        lower(trim(regexp_replace((regexp_matches(candidate_name,'[^ ]*$'))[1],'\*',''))) as LastName 
        from data_ingest.casos__california_candidate_statewide_election_results
        where county_name = 'State Totals' and contest_name <> 'President' and contest_name <> 'US Senate - 1') A
)

select
  E.FirstName as election_fn, E.LastName as election_ln, E.candidate_name as election_name, E.vote_total, E.is_winner,
  D.FirstName as donation_fn, D.LastName as donation_ln, D.candidate_name, D.total_transaction
from candidate_donations D, election_results E
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head()

In [ ]:
results = results.dropna()

In [ ]:
results.count()

In [ ]:
import numpy as np

In [ ]:
results['jwscore_fn'] = np.vectorize(distance.get_jaro_distance)(results.election_fn, results.donation_fn)
results['jwscore_ln'] = np.vectorize(distance.get_jaro_distance)(results.election_ln, results.donation_ln)

In [ ]:
results.head(5)

In [ ]:
results[results['jwscore_fn'] > 0.9]

Though more investigation can be conducted, let's save those that have a score above 0.9 for both first name and last name

In [ ]:
match_90 = results[(results['jwscore_fn'] >= 0.9) & (results['jwscore_ln'] >=0.9)]

Then we can explore which non-perfect matches we are included and evaluate if we agree with them

In [ ]:
match_90[(match_90['jwscore_fn'] < 1) | (match_90['jwscore_ln'] < 1)].head(20)

This list highlights some successes with these thresholds but also highlights some areas that could be investigated further (e.g. names in brackets or quotes)

In [ ]:
len(match_90)

In [ ]:
len(results[(results['jwscore_fn'] > 0.95) & (results['jwscore_ln'] > 0.95)])

In [ ]:
len(results[(results['jwscore_fn'] == 1) & (results['jwscore_ln'] == 1)])

In [ ]:
len(results[(results['jwscore_fn'] > 0.85) & (results['jwscore_ln'] > 0.85)])

In [ ]:
match_85 = results[(results['jwscore_fn'] > 0.85) & (results['jwscore_ln'] > 0.85)]

In [ ]:
match_85[(match_85['jwscore_fn'] < 1) | (match_85['jwscore_ln'] < 1)].head(20)

In [ ]:
match_85_90 = results[(results['jwscore_fn'] > 0.85) & (results['jwscore_ln'] > 0.9)]

In [ ]:
match_85_90[(match_85_90['jwscore_fn'] < 1) | (match_85_90['jwscore_ln'] < 1)].head(20)

In [ ]:
len(match_85_90)